
# Train your faultSeg model with synthetic datasets

it is a binary segmentation problem

the synthetic 3D seismic and corresponding fault images are in the folder ./data/train/ 

## Train with data generator

In [ ]:
from faultSeg_classes import DataGenerator

# training image dimensions
n1, n2, n3 = 128, 128, 128
params = {'batch_size': 1,
          'dim':(n1,n2,n3),
          'n_channels': 1,
          'shuffle': True}

tdpath = 'data/train/seis/'
tfpath = 'data/train/fault/'

vdpath = 'data/validation/seis/'
vfpath = 'data/validation/fault/'
tdata_IDs = range(200)
vdata_IDs = range(20)
training_generator   = DataGenerator(dpath=tdpath,fpath=tfpath,data_IDs=tdata_IDs,**params)
validation_generator = DataGenerator(dpath=vdpath,fpath=vfpath,data_IDs=vdata_IDs,**params)

In [ ]:
from unet3 import *
from keras import callbacks

K.set_image_data_format('channels_last')
model_name = 'fault'
model_dir     = os.path.join('check', model_name)
csv_fn        = os.path.join(model_dir, 'train_log.csv')
checkpoint_fn = os.path.join(model_dir, 'checkpoint.{epoch:02d}.hdf5')

model = unet()
checkpointer = callbacks.ModelCheckpoint(filepath=checkpoint_fn, verbose=1, save_best_only=False)
csv_logger  = callbacks.CSVLogger(csv_fn, append=True, separator=';')
tensorboard = callbacks.TensorBoard(log_dir=model_dir, histogram_freq=0, batch_size=2,
                                        write_graph=True, write_grads=True, write_images=True)
history = model.fit_generator(
                        generator=training_generator,
                        validation_data=validation_generator,
                        epochs=25,verbose=1,callbacks=[checkpointer, csv_logger, tensorboard])

print(history)
# serialize model to JSON
model_json = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model3.h5")
print("Saved model to disk") 

In [ ]:
# list all data in history
print(history.history.keys())
fig = plt.figure(figsize=(10,6))

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['train', 'test'], loc='center right',fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.tick_params(axis='both', which='minor', labelsize=18)
plt.show()

# summarize history for loss
fig = plt.figure(figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['train', 'test'], loc='center right',fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.tick_params(axis='both', which='minor', labelsize=18)
plt.show()

In [ ]:
from keras.models import load_model
from unet3 import *

# load json and create model 
json_file = open('model3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("check4/fault/checkpoint.25.hdf5")
print("Loaded model from disk")


In [ ]:
import matplotlib.pyplot as plt
gx,m1,m2,m3 = np.fromfile("../efault/data/trainx/seis/66.dat",dtype=np.single),128,128,128
gx = gx-np.min(gx)
gx = gx/np.max(gx)
gx = gx*255
k = 50
x = np.reshape(gx,(1,128,128,128,1))
Y = loaded_model.predict(x,verbose=1)
print(Y.shape)

# Y1 = Y[0]
# Y2 = Y[1]
# Y3 = Y[2]
# Y4 = Y[3]
# Y5 = Y[4]
#Y6 = Y[5]
fig = plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(x[0,k,:,:,0]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(Y[0,k,:,:,0]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
gy = Y[0,:,:,:,0]
gy.tofile("../efault/data/prediction/fake/"+"fp.dat",format="%4")

In [ ]:
from scipy.ndimage import gaussian_filter
gx,m1,m2,m3 = np.fromfile("../efault//data/prediction/f3d/ggl.dat",dtype=np.single),512,384,128

os = 64
c1 = np.round((m1+os)/(128-os)+0.5)
c2 = np.round((m2+os)/(128-os)+0.5)
c3 = np.round((m3+os)/(128-os)+0.5)
c1 = int(c1)
c2 = int(c2)
c3 = int(c3)
n1 = (128-os)*c1+os
n2 = (128-os)*c2+os
n3 = (128-os)*c3+os
gx = np.reshape(gx,(m1,m2,m3))
gp = np.zeros((n1,n2,n3),dtype=np.single)
gy = np.zeros((n1,n2,n3),dtype=np.single)
mk = np.zeros((n1,n2,n3),dtype=np.single)
gs = np.zeros((1,128,128,128,1),dtype=np.single)
gp[0:m1,0:m2,0:m3]=gx

for k1 in range(c1):
    for k2 in range(c2):
        for k3 in range(c3):
            b1 = k1*128-k1*os
            e1 = b1+128
            b2 = k2*128-k2*os
            e2 = b2+128
            b3 = k3*128-k3*os
            e3 = b3+128
            gs[0,:,:,:,0]=gp[b1:e1,b2:e2,b3:e3]
            gs = gs-np.min(gs)
            gs = gs/np.max(gs)
            gs = gs*255
            Y = loaded_model.predict(gs,verbose=1)
            Y = np.array(Y)
            gy[b1:e1,b2:e2,b3:e3]= gy[b1:e1,b2:e2,b3:e3]+Y[0,:,:,:,0]
            mk[b1:e1,b2:e2,b3:e3]= mk[b1:e1,b2:e2,b3:e3]+1
gy = gy/mk
gy = gy[0:m1,0:m2,0:m3]
gy.tofile("../efault/data/prediction/f3d/"+"fg2.dat",format="%4")
#gy = np.resize(gy,(m1,m2,m3))
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)


In [ ]:
from scipy.ndimage import gaussian_filter
gx,m1,m2,m3 = np.fromfile("../efault/data/prediction/clyde/gxl.dat",dtype=np.single),256,768,384

gx = np.reshape(gx,(m1,m2,m3))
gs = np.zeros((1, 128, 128, 128, 1),dtype=np.single)
gy = np.zeros((m1, m2, m3),dtype=np.single)
k=50
for k1 in range(2):
    for k2 in range(6):
        for k3 in range(3):
            b1 = k1*128
            e1 = b1+128
            b2 = k2*128
            e2 = b2+128
            b3 = k3*128
            e3 = b3+128
            gs[0,:,:,:,0]=gx[b1:e1,b2:e2,b3:e3]
            gs = gs-np.min(gs)
            gs = gs/np.max(gs)
            gs = gs*255
            gx[b1:e1,b2:e2,b3:e3]=gs[0,:,:,:,0]
            Y = loaded_model.predict(gs,verbose=1)
            Y = np.array(Y)
            gy[b1:e1,b2:e2,b3:e3]=Y[0,:,:,:,0]
gy.tofile("../efault/data/prediction/clyde/"+"fp.dat",format="%4")
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)


In [ ]:
from scipy.ndimage import gaussian_filter
from scipy.misc import imresize
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import griddata
gx,m1,m2,m3 = np.fromfile("../efault/data/prediction/clyde/gxl.dat",dtype=np.single),256,768,384
os = 64
c1 = np.round((m1+os)/(128-os)+0.5)
c2 = np.round((m2+os)/(128-os)+0.5)
c3 = np.round((m3+os)/(128-os)+0.5)
c1 = int(c1)
c2 = int(c2)
c3 = int(c3)
n1 = (128-os)*c1+os
n2 = (128-os)*c2+os
n3 = (128-os)*c3+os
gx = np.reshape(gx,(m1,m2,m3))
gp = np.zeros((n1,n2,n3),dtype=np.single)
gy = np.zeros((n1,n2,n3),dtype=np.single)
mk = np.zeros((n1,n2,n3),dtype=np.single)

gp[0:m1,0:m2,0:m3]=gx

for k1 in range(c1):
    for k2 in range(c2):
        for k3 in range(c3):
            b1 = k1*128-k1*os
            e1 = b1+128
            b2 = k2*128-k2*os
            e2 = b2+128
            b3 = k3*128-k3*os
            e3 = b3+128
            gs[0,:,:,:,0]=gp[b1:e1,b2:e2,b3:e3]
            gs = gs-np.min(gs)
            gs = gs/np.max(gs)
            gs = gs*255
            Y = loaded_model.predict(gs,verbose=1)
            Y = np.array(Y)
            gy[b1:e1,b2:e2,b3:e3]= gy[b1:e1,b2:e2,b3:e3]+Y[0,:,:,:,0]
            mk[b1:e1,b2:e2,b3:e3]= mk[b1:e1,b2:e2,b3:e3]+1
gy = gy/mk
gy = gy[0:m1,0:m2,0:m3]
#gy = np.resize(gy,(m1,m2,m3))
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)


In [ ]:
from scipy.ndimage import gaussian_filter
from scipy.misc import imresize
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import griddata
gx,m1,m2,m3 = np.fromfile("../efault/data/prediction/crf/gxl.dat",dtype=np.single),825,920,210
os = 64
c1 = np.round((m1+os)/(128-os)+0.5)
c2 = np.round((m2+os)/(128-os)+0.5)
c3 = np.round((m3+os)/(128-os)+0.5)
c1 = int(c1)
c2 = int(c2)
c3 = int(c3)
n1 = (128-os)*c1+os
n2 = (128-os)*c2+os
n3 = (128-os)*c3+os
gx = np.reshape(gx,(m1,m2,m3))
gp = np.zeros((n1,n2,n3),dtype=np.single)
gy = np.zeros((n1,n2,n3),dtype=np.single)
mk = np.zeros((n1,n2,n3),dtype=np.single)

gp[0:m1,0:m2,0:m3]=gx

for k1 in range(c1):
    for k2 in range(c2):
        for k3 in range(c3):
            b1 = k1*128-k1*os
            e1 = b1+128
            b2 = k2*128-k2*os
            e2 = b2+128
            b3 = k3*128-k3*os
            e3 = b3+128
            gs[0,:,:,:,0]=gp[b1:e1,b2:e2,b3:e3]
            gs = gs-np.min(gs)
            gs = gs/np.max(gs)
            gs = gs*255
            Y = loaded_model.predict(gs,verbose=1)
            Y = np.array(Y)
            gy[b1:e1,b2:e2,b3:e3]= gy[b1:e1,b2:e2,b3:e3]+Y[0,:,:,:,0]
            mk[b1:e1,b2:e2,b3:e3]= mk[b1:e1,b2:e2,b3:e3]+1
gy = gy/mk
gy = gy[0:m1,0:m2,0:m3]
gy.tofile("../efault/data/prediction/crf/"+"fp.dat",format="%4")

#gy = np.resize(gy,(m1,m2,m3))
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[:,k,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
fig = plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
imgplot1 = plt.imshow(np.transpose(gx[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.subplot(1, 2, 2)
imgplot2 = plt.imshow(np.transpose(gy[k,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)